In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20181001051905-0002


In [2]:
#get item table from source
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

In [3]:
#convert item table from html into pandas dataframe
df = pd.read_html(str(soup.table),header=0)[0]

In [4]:
#drop any rows that have value 'Not assigned' in column 'Borough'
df_clean = df[df['Borough'] != 'Not assigned' ].copy()
df_clean.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [5]:
#get index for all rows that have value 'Not assigned' in column 'Neighbourhood'
index_list = df_clean[df_clean['Neighbourhood'] == 'Not assigned'].index

In [5]:
#copy 'Borough' value into 'Neighbourhood' with the same index
for i in index_list:
    df_clean.loc[i, 'Neighbourhood'] = df_clean.loc[i, 'Borough']

df_clean.head(10)

NameError: name 'index_list' is not defined

In [7]:
#group the dataframe using value in 'Postcode' and 'Borough' and join 'Neighboorhood'
df_clean = df_clean.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_clean.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [9]:
df_clean.shape

(103, 3)